In [1]:
import requests
import json
import pandas as pd
import numpy as np
import os

In [2]:
## Get Geolocations for federal states and districts
with open('C:/Users/simon/Jupyter/Covid/Dashboard/landkreise_simplify200.geojson', 'r') as read_file:
    geo_kreis = json.load(read_file)

with open('C:/Users/simon/Jupyter/Covid/Dashboard/bundeslaender_simplify200.geojson', 'r') as read_file:
    geo_land = json.load(read_file)

In [3]:
## Get population stats for federal states and districts
df_einwohner_kreis = pd.read_csv('C:/Users/simon/Jupyter/Covid/Dashboard/Einwohner_kreis.csv', encoding = 'ISO-8859-1', sep=';', dtype= {'RS': str, 'Kreis': str, 'Einwohner': int})

df_einwohner_land = pd.read_csv('C:/Users/simon/Jupyter/Covid//Dashboard/Einwohner_land.csv', encoding = 'ISO-8859-1', sep=';', dtype= {'RS': str, 'Kreis': str, 'Einwohner': int})
df_einwohner_land.head()

,Bundesland,Einwohner
0,Baden-Württemberg,11100394
1,Bayern,13124737
2,Berlin,3669491
3,Brandenburg,2521893
4,Bremen,681202


In [4]:
## Get Covid stats from RKI
covid_url = 'https://opendata.arcgis.com/datasets/dd4580c810204019a7b8eb3e0b329dd6_0.geojson'
r = requests.get(covid_url)
r.status_code

200

In [5]:
a = json.loads(r.content)
df = pd.json_normalize(a['features'])

df.head()

,type,geometry,properties.ObjectId,properties.IdBundesland,properties.Bundesland,properties.Landkreis,properties.Altersgruppe,properties.Geschlecht,properties.AnzahlFall,properties.AnzahlTodesfall,properties.Meldedatum,properties.IdLandkreis,properties.Datenstand,properties.NeuerFall,properties.NeuerTodesfall,properties.Refdatum,properties.NeuGenesen,properties.AnzahlGenesen,properties.IstErkrankungsbeginn,properties.Altersgruppe2
0,Feature,None,1,1,Schleswig-Holstein,SK Flensburg,A00-A04,M,1,0,2020-09-30T00:00:00Z,01001,"22.05.2021, 00:00 Uhr",0,-9,2020-09-30T00:00:00Z,0,1,0,Nicht übermittelt
1,Feature,None,2,1,Schleswig-Holstein,SK Flensburg,A00-A04,M,1,0,2020-10-29T00:00:00Z,01001,"22.05.2021, 00:00 Uhr",0,-9,2020-10-29T00:00:00Z,0,1,0,Nicht übermittelt
2,Feature,None,3,1,Schleswig-Holstein,SK Flensburg,A00-A04,M,1,0,2020-11-03T00:00:00Z,01001,"22.05.2021, 00:00 Uhr",0,-9,2020-11-03T00:00:00Z,0,1,0,Nicht übermittelt
3,Feature,None,4,1,Schleswig-Holstein,SK Flensburg,A00-A04,M,1,0,2020-11-20T00:00:00Z,01001,"22.05.2021, 00:00 Uhr",0,-9,2020-11-19T00:00:00Z,0,1,1,Nicht übermittelt
4,Feature,None,5,1,Schleswig-Holstein,SK Flensburg,A00-A04,M,1,0,2020-11-23T00:00:00Z,01001,"22.05.2021, 00:00 Uhr",0,-9,2020-11-18T00:00:00Z,0,1,1,Nicht übermittelt


In [6]:
## Clean-up and data preparation
df.columns = df.columns.str.replace('properties.', '')
df = df[['IdBundesland', 'Bundesland', 'Landkreis', 'IdLandkreis', 'AnzahlFall', 'Altersgruppe', 'AnzahlTodesfall', 'Geschlecht', 'Meldedatum']]
df['Meldedatum'] = df['Meldedatum'].str.replace('T00:00:00Z', '')
df['Meldedatum'] = pd.to_datetime(df['Meldedatum'])

df = df.merge(df_einwohner_kreis, how='left', left_on='IdLandkreis', right_on='RS')
df = df.merge(df_einwohner_land, how='left', left_on='Bundesland', right_on='Bundesland') 

df_gender = pd.get_dummies(df['Geschlecht']).mul(df['AnzahlFall'],0)
df_gender.columns = ['Sex_M', 'Sex_F', 'Sex_Unbekannt']
df = df.join(df_gender)

df_age = pd.get_dummies(df['Altersgruppe']).mul(df['AnzahlFall'],0)
df = df.join(df_age)



df.drop(['RS', 'Kreis'], axis=1, inplace=True)

df.head()

,IdBundesland,Bundesland,Landkreis,IdLandkreis,AnzahlFall,Altersgruppe,AnzahlTodesfall,Geschlecht,Meldedatum,Einwohner_x,...,Sex_M,Sex_F,Sex_Unbekannt,A00-A04,A05-A14,A15-A34,A35-A59,A60-A79,A80+,unbekannt
0,1,Schleswig-Holstein,SK Flensburg,01001,1,A00-A04,0,M,2020-09-30,90164.0,...,1,0,0,1,0,0,0,0,0,0
1,1,Schleswig-Holstein,SK Flensburg,01001,1,A00-A04,0,M,2020-10-29,90164.0,...,1,0,0,1,0,0,0,0,0,0
2,1,Schleswig-Holstein,SK Flensburg,01001,1,A00-A04,0,M,2020-11-03,90164.0,...,1,0,0,1,0,0,0,0,0,0
3,1,Schleswig-Holstein,SK Flensburg,01001,1,A00-A04,0,M,2020-11-20,90164.0,...,1,0,0,1,0,0,0,0,0,0
4,1,Schleswig-Holstein,SK Flensburg,01001,1,A00-A04,0,M,2020-11-23,90164.0,...,1,0,0,1,0,0,0,0,0,0


In [7]:
df_bund = df.groupby(by=['Meldedatum']).agg({
    'AnzahlFall':'sum',
    'AnzahlTodesfall': 'sum',
    'Sex_M': 'sum',
    'Sex_F': 'sum',
    'Sex_Unbekannt': 'sum',
    'A00-A04': 'sum',
    'A05-A14': 'sum',
    'A15-A34': 'sum',
    'A35-A59': 'sum',
    'A60-A79': 'sum',
    'A80+': 'sum',
    'unbekannt': 'sum'
})


#df_bund.set_index('Region', append=True, inplace=True)
#df_bund = df_bund.swaplevel('Region', 'Meldedatum')

df_bund['Fall_Last7'] = df_bund['AnzahlFall'].rolling(min_periods=1, window=7).sum()
df_bund['Fall_MA7'] = df_bund['AnzahlFall'].rolling(7).mean()
df_bund['R7'] = (df_bund['Fall_MA7'].pct_change(periods=4) + 1).round(2)

df_bund['Einwohner'] = df_einwohner_land['Einwohner'].sum()
df_bund['Fall_100K'] = ((df_bund['Fall_Last7'] / df_bund['Einwohner']) * 100000).round(2)

df_bund.reset_index(inplace=True)
df_bund['Region'] = 'Deutschland'
df_bund['Typ'] = 'Nation'


df_bund.tail()

,Meldedatum,AnzahlFall,AnzahlTodesfall,Sex_M,Sex_F,Sex_Unbekannt,A00-A04,A05-A14,A15-A34,A35-A59,A60-A79,A80+,unbekannt,Fall_Last7,Fall_MA7,R7,Einwohner,Fall_100K,Region,Typ
465,2021-05-17,5265,11,2754,2472,39,191,632,1610,1992,661,172,7,68755.0,9822.142857,0.83,83166711,82.67,Deutschland,Nation
466,2021-05-18,12316,13,6183,6036,97,501,1542,3757,4784,1466,262,4,64259.0,9179.857143,0.85,83166711,77.27,Deutschland,Nation
467,2021-05-19,11214,12,5732,5371,111,448,1512,3501,4071,1355,316,11,58982.0,8426.000000,0.82,83166711,70.92,Deutschland,Nation
468,2021-05-20,8034,1,3953,3960,121,365,1099,2566,2843,943,210,8,57324.0,8189.142857,0.81,83166711,68.93,Deutschland,Nation
469,2021-05-21,5750,3,2792,2854,104,303,914,1875,1918,574,154,12,55444.0,7920.571429,0.81,83166711,66.67,Deutschland,Nation


In [8]:
df_land = df.groupby(by=['Bundesland', 'Meldedatum']).agg({
    'IdBundesland': 'first',
    'AnzahlFall': 'sum',
    'Einwohner_y': 'first',
    'AnzahlTodesfall': 'sum',
    'Sex_M': 'sum',
    'Sex_F': 'sum',
    'Sex_Unbekannt': 'sum',
    'A00-A04': 'sum',
    'A05-A14': 'sum',
    'A15-A34': 'sum',
    'A35-A59': 'sum',
    'A60-A79': 'sum',
    'A80+': 'sum',
    'unbekannt': 'sum'
})

df_land.rename(columns={'Einwohner_y': 'Einwohner'}, inplace=True)

#df_land.index.names = ['Region', 'Meldedatum']

df_land['Fall_MA4'] = df_land.groupby(level=0, group_keys=True)['AnzahlFall'].rolling(4).mean().values
df_land['Fall_MA7'] = df_land.groupby(level=0, group_keys=True)['AnzahlFall'].rolling(7).mean().values

df_land['R4'] = df_land['Fall_MA4'].pct_change(periods=4) + 1
df_land['R7'] = (df_land['Fall_MA7'].pct_change(periods=4) + 1).round(2)

df_land['Fall_Last7'] = df_land['AnzahlFall'].rolling(min_periods=1, window=7).sum()
df_land['Fall_100K'] = ((df_land['Fall_Last7'] / df_land['Einwohner']) * 100000).round(2)

#

df_land.reset_index(inplace=True)
df_land['Region'] = df_land['Bundesland']
df_land['Typ'] = 'Bundesland'

df_land.tail()

,Bundesland,Meldedatum,IdBundesland,AnzahlFall,Einwohner,AnzahlTodesfall,Sex_M,Sex_F,Sex_Unbekannt,A00-A04,...,A80+,unbekannt,Fall_MA4,Fall_MA7,R4,R7,Fall_Last7,Fall_100K,Region,Typ
7019,Thüringen,2021-05-17,16,261,2133378,1,134,126,1,9,...,15,1,251.50,376.142857,0.514052,0.81,2633.0,123.42,Thüringen,Bundesland
7020,Thüringen,2021-05-18,16,522,2133378,2,247,270,5,20,...,23,0,305.50,360.857143,0.632178,0.86,2526.0,118.40,Thüringen,Bundesland
7021,Thüringen,2021-05-19,16,488,2133378,1,257,227,4,17,...,24,0,356.25,334.142857,0.896790,0.85,2339.0,109.64,Thüringen,Bundesland
7022,Thüringen,2021-05-20,16,288,2133378,1,135,153,0,14,...,8,1,389.75,329.142857,1.459738,0.85,2304.0,108.00,Thüringen,Bundesland
7023,Thüringen,2021-05-21,16,239,2133378,0,118,116,5,7,...,12,0,384.25,319.571429,1.527833,0.85,2237.0,104.86,Thüringen,Bundesland


In [9]:
df_kreis = df.groupby(by=['Landkreis', 'Meldedatum']).agg({
    'IdLandkreis': 'first',
    'Bundesland': 'first',
    'AnzahlFall': 'sum',
    'Einwohner_x': 'first',
    'AnzahlTodesfall': 'sum',
    'Sex_M': 'sum',
    'Sex_F': 'sum',
    'Sex_Unbekannt': 'sum',
    'A00-A04': 'sum',
    'A05-A14': 'sum',
    'A15-A34': 'sum',
    'A35-A59': 'sum',
    'A60-A79': 'sum',
    'A80+': 'sum',
    'unbekannt': 'sum'
})
    
df_kreis.rename(columns={'Einwohner_x': 'Einwohner'}, inplace=True)


#df_kreis.index.names = ['Region', 'Meldedatum']

df_kreis['Fall_MA4'] = df_kreis.groupby(level=0, group_keys=True)['AnzahlFall'].rolling(4).mean().values
df_kreis['Fall_MA7'] = df_kreis.groupby(level=0, group_keys=True)['AnzahlFall'].rolling(7).mean().values

df_kreis['R4'] = df_kreis['Fall_MA4'].pct_change(periods=4) + 1
df_kreis['R7'] = (df_kreis['Fall_MA7'].pct_change(periods=4) + 1).round(2)

df_kreis['Fall_Last7'] = df_kreis['AnzahlFall'].rolling(min_periods=1, window=7).sum()
df_kreis['Fall_100K'] = ((df_kreis['Fall_Last7'] / df_kreis['Einwohner']) * 100000).round(2)

df_kreis.reset_index(inplace=True)
df_kreis['Region'] = df_kreis['Landkreis']
df_kreis['Typ'] = 'Landkreis'

df_kreis.tail()

,Landkreis,Meldedatum,IdLandkreis,Bundesland,AnzahlFall,Einwohner,AnzahlTodesfall,Sex_M,Sex_F,Sex_Unbekannt,...,A80+,unbekannt,Fall_MA4,Fall_MA7,R4,R7,Fall_Last7,Fall_100K,Region,Typ
137731,Städteregion Aachen,2021-05-17,05334,Nordrhein-Westfalen,36,557026.0,0,21,15,0,...,0,0,75.00,97.714286,0.691244,1.09,684.0,122.79,Städteregion Aachen,Landkreis
137732,Städteregion Aachen,2021-05-18,05334,Nordrhein-Westfalen,143,557026.0,0,80,63,0,...,2,0,82.50,99.000000,0.663984,1.11,693.0,124.41,Städteregion Aachen,Landkreis
137733,Städteregion Aachen,2021-05-19,05334,Nordrhein-Westfalen,84,557026.0,0,48,36,0,...,3,0,82.25,91.714286,0.734375,0.98,642.0,115.25,Städteregion Aachen,Landkreis
137734,Städteregion Aachen,2021-05-20,05334,Nordrhein-Westfalen,83,557026.0,0,43,40,0,...,1,0,86.50,87.142857,0.912929,0.87,610.0,109.51,Städteregion Aachen,Landkreis
137735,Städteregion Aachen,2021-05-21,05334,Nordrhein-Westfalen,44,557026.0,0,22,22,0,...,0,0,88.50,77.285714,1.180000,0.79,541.0,97.12,Städteregion Aachen,Landkreis


In [10]:
## Final dataset
df_merged = pd.concat([df_bund, df_land, df_kreis])

## The geolocations for the districts of Berlin are not included in the dataset, 
## to have Berlin displayed on the choropleth map, I substitute the values for the federal state of Berlin.
df_merged.loc[df_merged['Bundesland'] == 'Berlin', 'IdLandkreis'] = '11000'

df_merged.tail()

,Meldedatum,AnzahlFall,AnzahlTodesfall,Sex_M,Sex_F,Sex_Unbekannt,A00-A04,A05-A14,A15-A34,A35-A59,...,Einwohner,Fall_100K,Region,Typ,Bundesland,IdBundesland,Fall_MA4,R4,Landkreis,IdLandkreis
137731,2021-05-17,36,0,21,15,0,2,9,8,10,...,557026.0,122.79,Städteregion Aachen,Landkreis,Nordrhein-Westfalen,NaN,75.00,0.691244,Städteregion Aachen,05334
137732,2021-05-18,143,0,80,63,0,6,19,49,55,...,557026.0,124.41,Städteregion Aachen,Landkreis,Nordrhein-Westfalen,NaN,82.50,0.663984,Städteregion Aachen,05334
137733,2021-05-19,84,0,48,36,0,3,14,26,31,...,557026.0,115.25,Städteregion Aachen,Landkreis,Nordrhein-Westfalen,NaN,82.25,0.734375,Städteregion Aachen,05334
137734,2021-05-20,83,0,43,40,0,8,13,23,32,...,557026.0,109.51,Städteregion Aachen,Landkreis,Nordrhein-Westfalen,NaN,86.50,0.912929,Städteregion Aachen,05334
137735,2021-05-21,44,0,22,22,0,4,7,18,12,...,557026.0,97.12,Städteregion Aachen,Landkreis,Nordrhein-Westfalen,NaN,88.50,1.180000,Städteregion Aachen,05334


In [11]:
if os.path.exists('covid_panel_backup.csv'):
    os.remove('covid_panel_backup.csv')

In [12]:
os.rename('covid_panel.csv', 'covid_panel_backup.csv')

In [13]:
df_merged.to_csv('covid_panel.csv')